####**Name**:  Ahmed Abdo Amin Abdo
####**Student number**: 300327306

##Python Development

In [ ]:
!pip install cmake 'gym' scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import libraries

In [ ]:
import gym
from numpy import array
from math import inf
from numpy.linalg import norm
from IPython.display import clear_output
from time import sleep
import numpy as np
import random
from time import sleep
import pandas as pd

## Setting the state of the environment

In [ ]:
def setState(env, agentRow =3 , agentColumn =1, sourceIndex =2, destination = 0):
  state = env.encode(agentRow, agentColumn, sourceIndex, destination) # (taxi row, taxi column, passenger index, destination
  return state

## simulating the steps for solving the environment 

In [ ]:
def print_frames(frames):
  for i, frame in enumerate(frames):
    clear_output(wait = True)
    print(frame['frame'])
    print(f'Timestep: {i+1}')
    print(f"State: {frame['state']}")
    print(f"Action: {frame['action']}")
    print(f"Reward: {frame['reward']}")
    sleep(.1)

## Setting brute-force function to Solve the environment without Reinforcement Learning

In [ ]:
def LR_bruteforce(env, state, maxPunishScore):
  env.render()
  #print("State:", state)
  env.s = state # set environment to illustration's state
  epochs = 0
  penalties, reward =0, 0
  frames = [] # for animation
  done = False
  while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    if reward == maxPunishScore:
       penalties += 1
    # Put each rendered frame into dict for animation
    frames.append({
          'frame': env.render(mode = 'ansi'),
          'state': state,
          'action': action,
          'reward': reward
          })
    epochs +=1
  print(f"TimeSteps taken: {format(epochs)}")
  print(f"Penaltes incurred: {format(penalties)}")
  return frames

## Setting Q-learning function to Solve the environment with Reinforcement Learning

### Training the Agent

In [ ]:
def QlearningTrain(env, Talpha=0.1, Tgamma=0.6, Tepsilon=0.1):
  q_table = np.zeros([env.observation_space.n, env.action_space.n])
  # Hyperparameters
  ## the learning rate
  alpha = Talpha
  ## determines how much importance we want to give to future rewards.
  gamma = Tgamma
  ## trials increase, epsilon should decrease.
  epsilon = Tepsilon

  # for plotting metrics
  all_epochs = []
  all_penalties = []

  for i in range(1, 100001):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    done = False

    while not done:
      if random.uniform(0, 1) < epsilon:
        action = env.action_space.sample() # Explore action space
      else:
        action = np.argmax(q_table[state]) # Exploit learned values

      next_state, reward, done, info = env.step(action)
      old_value = q_table[state, action]
      next_max = np.max(q_table[next_state])
      new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
      q_table[state, action] = new_value
      state = next_state

      if reward == -10:
        penalties +=1

      epochs +=1
      if i % 100 == 0:
        clear_output(wait = True)
        print(f"Episode: {i}")
  
  print("Training finished. \n")
  return q_table

###Evaluating the agent

In [ ]:
def QlearningEvaluating(env, q_table, n= 100):
  total_epochs, total_penalties = 0, 0
  episodes = n

  for _ in range(episodes):
      state = env.reset()
      epochs, penalties, reward = 0, 0, 0
      
      done = False
      
      while not done:
          action = np.argmax(q_table[state])
          state, reward, done, info = env.step(action)

          if reward == -10:
              penalties += 1

          epochs += 1

      total_penalties += penalties
      total_epochs += epochs
  return (total_epochs / episodes)

##Showing some environments

In [ ]:
#from gym import envs
#env_names = [spec.id for spec in envs.registry.all()]
lstEnvs = ["Taxi-v3", "CliffWalking-v0", "FrozenLake-v1"]
for name in sorted(lstEnvs):
  print(name)



CliffWalking-v0
FrozenLake-v1
Taxi-v3


## Solving the environment without Reinforcement Learning (brute-force)

In [ ]:
# get he environment
env = gym.make("Taxi-v3")
# setting the state of the environment
state = setState(env, 3, 1, 2, 0)
# Using  brute-force function to Solve the environment 
frames = LR_bruteforce(env, state, -10)
# simulating the steps of brute-force for solving the environment
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 4157
State: 0
Action: 5
Reward: 20


##Solving the environment with Q-learning (Reinforcement Learning)

In [ ]:
# the number of range in alpha & gamma & epsilon
n = 2
alpha =list(np.linspace(0.1,0.2,n))
gamma = list(np.linspace(0.6,0.3,n))
epsilon = list(np.linspace(0.1,0.2,n))

for Ta in alpha:
  for Tg in gamma:
    for Te in epsilon:
      print(f'{Ta}  {Tg} {Te}')

0.1  0.6 0.1
0.1  0.6 0.2
0.1  0.3 0.1
0.1  0.3 0.2
0.2  0.6 0.1
0.2  0.6 0.2
0.2  0.3 0.1
0.2  0.3 0.2


In [ ]:
# Setting Lists for All probabilities of alpha, gamma, epsilon
alphaLst = []
gammaLst = []
epsilonLst = []
Average_timesteps_Lst = []

env = gym.make("Taxi-v3")

for Ta in alpha:
  for Tg in gamma:
    for Te in epsilon:
      q_tabletrain = QlearningTrain(env, Ta, Tg, Te)
      Average_timesteps = QlearningEvaluating(env, q_tabletrain, 100)
      alphaLst.append(Ta)
      gammaLst.append(Tg)
      epsilonLst.append(Te)
      Average_timesteps_Lst.append(Average_timesteps)

df = pd.DataFrame()
df["Alpha"] = alphaLst
df["Gamma"] = gammaLst
df["epsilon"] = epsilonLst
df["Average timesteps per episode"] = Average_timesteps_Lst
df


Episode: 100000
Training finished. 



,Alpha,Gamma,epsilon,Average timesteps per episode
0,0.1,0.6,0.1,12.64
1,0.1,0.6,0.2,12.91
2,0.1,0.3,0.1,12.91
3,0.1,0.3,0.2,13.30
4,0.2,0.6,0.1,12.26
5,0.2,0.6,0.2,12.85
6,0.2,0.3,0.1,12.95
7,0.2,0.3,0.2,13.36


### Getting the value of alpha, gamma,	and epsilon to get the smallest average timesteps per episode.

In [ ]:
#df['Average timesteps per episode'].min()
pd.DataFrame(df.min(axis = 0))

,0
Alpha,0.10
Gamma,0.30
epsilon,0.10
Average timesteps per episode,12.26
